# 한글 손글씨 인식 서비스 만들기 

In [2]:
import torch
from transformers import VisionEncoderDecoderModel

In [5]:
# ver 1

import os
import gradio as gr
from PIL import Image
import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

class TrOCRInferencer:
    def __init__(self):
        print("[info] init TrOCR Inferencer")
        # 한글 손글씨 인식용 모델 및 프로세서 로드
        self.processor = TrOCRProcessor.from_pretrained("team-lucid/trocr-small-korean")
        self.model = VisionEncoderDecoderModel.from_pretrained("team-lucid/trocr-small-korean")
        
    def inference(self, image):
        # 이미지를 모델에 맞게 전처리
        pixel_values = self.processor(images=image, return_tensors='pt').pixel_values
        generated_ids = self.model.generate(pixel_values)
        generated_text = self.processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        
        return generated_text

inferencer = TrOCRInferencer()

def image_to_text(image):
    image = Image.fromarray(image).convert('RGB')  # 이미지를 RGB로 변환
    text = inferencer.inference(image)  # 텍스트 인식
    return text

# Gradio 앱 구성
with gr.Blocks() as app:
    gr.Markdown("# Handwritten Image OCR")

    with gr.Tab("Image upload"):
        image = gr.Image(label="Handwritten image file")
        output = gr.Textbox(label="Output Box")
        convert_btn = gr.Button("Convert")
        convert_btn.click(fn=image_to_text, inputs=image, outputs=output)

    with gr.Tab("Drawing"):
        gr.Markdown("# Handwritten Image OCR")
        sketchpad = gr.Sketchpad(
            label="Handwritten Sketchpad",
            shape=(600, 300),
            brush_radius=3,
            invert_colors=False,
        )
        output = gr.Textbox(label="Output Box")
        convert_btn = gr.Button("Convert")
        convert_btn.click(fn=image_to_text, inputs=sketchpad, outputs=output)

# Gradio 앱 실행
app.launch(inline=False, share=True)


[info] init TrOCR Inferencer
Running on local URL:  http://127.0.0.1:7870

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://8da38f0c6ddc38105f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [6]:
app.close()

Closing server running on port: 7870


### ver 1 결과 
small 은 너무 작아서 한글 인식을 못한다. 

In [7]:
# ver 2

import gradio as gr
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, AutoTokenizer
import unicodedata
from PIL import Image

class TrOCRInferencer:
    def __init__(self):
        print("[info] init TrOCR Inferencer")
        # 한글 손글씨 인식용 모델, 프로세서, 토크나이저 로드
        self.processor = TrOCRProcessor.from_pretrained("ddobokki/ko-trocr")
        self.model = VisionEncoderDecoderModel.from_pretrained("ddobokki/ko-trocr")
        self.tokenizer = AutoTokenizer.from_pretrained("ddobokki/ko-trocr")
        
    def inference(self, image):
        # 이미지를 모델에 맞게 전처리
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        generated_ids = self.model.generate(pixel_values, max_length=64)
        generated_text = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        generated_text = unicodedata.normalize("NFC", generated_text)  # 한글 정규화 처리
        return generated_text

# TrOCR Inferencer 인스턴스 생성
inferencer = TrOCRInferencer()

# 이미지에서 텍스트를 추출하는 함수
def image_to_text(image):
    image = Image.fromarray(image).convert('RGB')  # 이미지를 RGB로 변환
    text = inferencer.inference(image)  # 텍스트 인식
    return text

# Gradio 앱 구성
with gr.Blocks() as app:
    gr.Markdown("# Handwritten Korean OCR")

    with gr.Tab("Image upload"):
        image = gr.Image(label="Handwritten image file")
        output = gr.Textbox(label="Output Box")
        convert_btn = gr.Button("Convert")
        convert_btn.click(fn=image_to_text, inputs=image, outputs=output)

    with gr.Tab("Drawing"):
        gr.Markdown("# Handwritten Image OCR")
        sketchpad = gr.Sketchpad(
            label="Handwritten Sketchpad",
            shape=(600, 300),
            brush_radius=3,
            invert_colors=False,
        )
        output = gr.Textbox(label="Output Box")
        convert_btn = gr.Button("Convert")
        convert_btn.click(fn=image_to_text, inputs=sketchpad, outputs=output)

# Gradio 앱 실행
app.launch(inline=False, share=True)


[info] init TrOCR Inferencer


/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Running on local URL:  http://127.0.0.1:7870
IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://31d371264755938028.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
app.close()

### ver 2 결과 
오히려 large 로 하니깐 한글 인식이 잘된다. 